### Question 3: Train a Pure CNN with less than 10000 trainable parameters using the MNIST Dataset having minimum validation accuracy of 99.40%


### Tensorflow Implementation

In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape and normalize input data
x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0

# One-hot encode the target labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Define the CNN architecture
model = Sequential([
    Conv2D(8, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(16, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
#    Dropout(0.5),
    Flatten(),
    Dense(20, activation='relu'),
    Dense(10, activation='softmax')
])


In [3]:
# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 8)         80        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 8)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 16)        1168      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 16)         0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 5, 5, 16)          0         
                                                                 
 flatten (Flatten)           (None, 400)               0

In [4]:
# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, epochs=25, batch_size=32, validation_data=(x_test, y_test))

# Evaluate the model on the test set
_, accuracy = model.evaluate(x_test, y_test)
print("Test Accuracy:", accuracy*100)


Epoch 1/25
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4254 - accuracy: 0.8634 - val_loss: 0.1029 - val_accuracy: 0.9701
Epoch 2/25
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1621 - accuracy: 0.9500 - val_loss: 0.0669 - val_accuracy: 0.9788
Epoch 3/25
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1312 - accuracy: 0.9589 - val_loss: 0.0560 - val_accuracy: 0.9841
Epoch 4/25
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1148 - accuracy: 0.9646 - val_loss: 0.0459 - val_accuracy: 0.9852
Epoch 5/25
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1072 - accuracy: 0.9673 - val_loss: 0.0419 - val_accuracy: 0.9879
Epoch 6/25
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1014 - accuracy: 0.9686 - val_loss: 0.0408 - val_accuracy: 0.9875
Epoch 7/25
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0955 - accuracy: 0.9698 - val_loss: 0.0378 - val_accuracy:

### Pytorch Implementation

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data.sampler import SubsetRandomSampler

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load MNIST dataset
train_dataset = MNIST(root="./data", train=True, download=True, transform=ToTensor())
test_dataset = MNIST(root="./data", train=False, download=True, transform=ToTensor())

# Define indices for train-validation split
num_samples = len(train_dataset)
indices = list(range(num_samples))
split = int(0.8 * num_samples)  # 80% train, 20% validation
train_indices, val_indices = indices[:split], indices[split:]

# Create data loaders
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(train_dataset, batch_size=32, sampler=train_sampler)
val_loader = DataLoader(train_dataset, batch_size=32, sampler=val_sampler)
test_loader = DataLoader(test_dataset, batch_size=32)

# Define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, stride=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, stride=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(16 * 5 * 5, 20)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(20, 10)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = self.dropout(x)
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        return x

# Create the model instance
model = Net().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Training loop
num_epochs = 25
best_val_acc = 0.0

for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss = 0.0
    train_correct = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        train_correct += (predicted == labels).sum().item()

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_acc = 100.0 * train_correct / len(train_indices)

    # Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()

            loss = criterion(outputs, labels)
            val_loss += loss.item()

    val_acc = 100.0 * val_correct / len(val_indices)

    print(f"Epoch {epoch + 1}/{num_epochs}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

    # Save the model with the best validation accuracy
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model.pt")

# Load the best model weights
model.load_state_dict(torch.load("best_model.pt"))

# Evaluation on test set
model.eval()
test_correct = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_correct += (predicted == labels).sum().item()

test_acc = 100.0 * test_correct / len(test_dataset)

print(f"Test Accuracy: {test_acc:.2f}%")


Epoch 1/25: Train Loss: 724.1251, Train Acc: 84.61%, Val Loss: 50.9466, Val Acc: 96.24%
Epoch 2/25: Train Loss: 320.7783, Train Acc: 93.36%, Val Loss: 38.0851, Val Acc: 97.05%
Epoch 3/25: Train Loss: 264.5749, Train Acc: 94.45%, Val Loss: 32.8299, Val Acc: 97.37%
Epoch 4/25: Train Loss: 235.4968, Train Acc: 95.10%, Val Loss: 27.4179, Val Acc: 97.88%
Epoch 5/25: Train Loss: 225.5769, Train Acc: 95.34%, Val Loss: 26.1155, Val Acc: 97.97%
Epoch 6/25: Train Loss: 213.9495, Train Acc: 95.51%, Val Loss: 25.8241, Val Acc: 97.98%
Epoch 7/25: Train Loss: 198.3514, Train Acc: 95.89%, Val Loss: 24.2367, Val Acc: 97.92%
Epoch 8/25: Train Loss: 191.2112, Train Acc: 96.03%, Val Loss: 23.0530, Val Acc: 98.05%
Epoch 9/25: Train Loss: 180.1479, Train Acc: 96.21%, Val Loss: 20.9298, Val Acc: 98.26%
Epoch 10/25: Train Loss: 166.4154, Train Acc: 96.40%, Val Loss: 23.4028, Val Acc: 98.22%
Epoch 11/25: Train Loss: 170.7314, Train Acc: 96.45%, Val Loss: 19.7236, Val Acc: 98.42%
Epoch 12/25: Train Loss: 163.4